In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import uniform_filter1d

## 1- Load the dataset from the simulation

This dataset contains horizontally-averaged quantities all stored in a simple text file

We first start by defining a couple of useful functions to read the input file and to make time averages

In [ ]:
def readProfile(filename):
        
    fid=open(filename,"r")
    
    # read the first line to get data names
    varnames=fid.readline().split()
    fid.close()
    
    nprof=np.size(varnames);
    
    # load the bulk of the file
    data=np.loadtxt(filename,skiprows=1)
    
    # store this in our data structure
    prof={}
    prof['z']=data[0,1:]
    prof['t']=data[1::nprof,0]

    i=0
    for name in varnames:
        prof[name]=np.transpose(data[1+i::nprof,1:])
        i=i+1
        
    return prof

# Compute time averages from profiles

def make_time_averages(prof,period):
    nt = prof['t'].size
    nz = prof['z'].size
    # compute the number of time averages we have in the series
    navg = (nt//period)
    ntot = navg * period
    avg = {} # Average value during #period
    start = {} # Start value at begining of #period
    for key in prof.keys():
        if prof[key].ndim == 2:
            field = np.reshape(prof[key][:,:ntot],(nz,navg,period))
            avg[key]=np.mean(field,axis=2)
            start[key]=field[:,:,0]
    start['t']=prof['t'][:ntot:period]
    avg['t']=start['t']
    avg['z']=prof['z']
    start['z']=prof['z']
    return avg,start

We load Idefix output in a dictionnary named "prof".

In [ ]:
prof=readProfile("timevol_2.dat")
prof.keys()

As you can see, the dataset contains many properties, such as the density "rho", the velocity field "vx", "vy", "vz", etc. These are all horizontally averaged quantities, stored as functions of $z$ and $t$. Let's plot the density "rho", and the horizontal components of the magnetic field "Bx" and "By":

In [ ]:
plt.figure(figsize=(13,4))
plt.pcolormesh(prof['t'],prof['z'],np.log10(prof['rho']),cmap='gnuplot')
plt.xlabel('t')
plt.ylabel('z')
plt.colorbar()
plt.title(r'$\rho$')

plt.figure(figsize=(13,4))
plt.pcolormesh(prof['t'],prof['z'],prof['bx'],cmap='seismic')
plt.xlabel('t')
plt.ylabel('z')
plt.colorbar()
plt.title('$B_x$')

plt.figure(figsize=(13,4))
plt.pcolormesh(prof['t'],prof['z'],prof['by'],cmap='seismic')
plt.xlabel('t')
plt.ylabel('z')
plt.colorbar()
plt.title('$B_y$')

In order to reduce the noise, we average our profile in a time window. In addition we compute the vertical gradient of the EMFs that will be useful later

In [ ]:
# We will make use of log(rho), so let's compute this now
prof['logrho']=np.log10(prof['rho'])

# Make some time average on a window
period=50
avg,start=make_time_averages(prof,period)

dz=prof['z'][1]-prof['z'][0]
avg['dzEx']=np.gradient(avg['Ex'],dz,axis=0)
avg['dzEy']=np.gradient(avg['Ey'],dz,axis=0)

Let's plot the time-average quantities, that you can compare to the raw outputs above

In [ ]:
plt.figure(figsize=(13,4))
plt.pcolormesh(avg['t'],avg['z'],avg['by'],cmap='seismic')
plt.xlabel('t')
plt.ylabel('z')
plt.title('$B_y$')

## 2- Transform this into a dataset for a neural network

### a- basic idea
We want to reproduce the butterfly diagram using an "effective" subgrid model. For this, we will use the horizontally averaged induction equation
$$
\partial_t \langle B_x\rangle =\partial_z \langle E_y\rangle
$$
$$
\partial_t \langle B_y\rangle =-q\Omega B_x-\partial_z \langle E_x \rangle
$$
We will need a "subgrid" model to know what's happening to $E_x$ and $E_y$, with $E=\langle \delta v\times\delta B\rangle$. 

We will make the hypothesis that $E_x(z)$ and $E_y(z)$ are functions of the values of $B_x,B_y,\rho$ at $z$ and at the immediate neighbouring cells of $z$: $z+dz$ and $z-dz$. Hence, the input of our network (named $X$ in the following) will be $B_x,B_y,\rho$ at $z-dz,z,z+dz$ and the outputs (named $Y$) will be $E_x,E_y$ at $z$.

### b- create the input set $X$ and output set $Y$

To avoid the initial transient of the simulations, we start sampling at $t_\mathrm{min}=100$. We organise $X$ and $Y$ in samples $s$

In [ ]:

Xkeylist=['bx','by','logrho'] # the list of input variables with which we work 
Ykeylist=['Ex','Ey'] # the list of variables we ought to guess
neighb=1 # number of neighbours in z
tmin=100
Xnvars=len(Xkeylist)
Ynvars=len(Ykeylist)
nt=np.sum(avg['t']>tmin)  # array size in t
nt_start=np.argwhere(avg['t']>tmin)[0][0]   # index of the first item
nz=avg['z'].size-2*neighb # remove the boundaries in z
X_total=np.zeros([nt*nz,3*Xnvars]) # 3* for the cell center+left neighbour+right neighbour
Y_total=np.zeros([nt*nz,Ynvars]) # 
nk=0 # number of the key

# fill X
for key in Xkeylist:
    pcentered=avg[key][neighb:-neighb,nt_start:].flatten()
    pm1=avg[key][0:-2*neighb,nt_start:].flatten()
    pp1=avg[key][2*neighb:,nt_start:].flatten()
    X_total[:,nk]=pcentered
    X_total[:,nk+Xnvars]=pm1
    X_total[:,nk+2*Xnvars]=pp1
    nk=nk+1
    
# fill Y
nk=0
Yscaling=np.zeros([Ynvars,2])
for key in Ykeylist:
    pcentered=avg[key][neighb:-neighb,nt_start:].flatten()
    Yscaling[nk,0]=np.mean(pcentered)  # Mean
    Yscaling[nk,1]=np.sqrt(np.mean((pcentered-Yscaling[nk,0])**2)) # RMS
    Y_total[:,nk]=pcentered
    nk=nk+1
    

Visualize the two components of the Y sample (i.e $E_x$ and $E_y$ of each sample)

In [ ]:
plt.figure()
plt.plot(Y_total[:,0])
plt.title("$E_x$")
plt.figure()
plt.plot(Y_total[:,1])
plt.title("$E_y$")

### c- Shuffle and split

The data has retained the vertical structure of the flow (see figures above!), so we must shuffle this to make a random training set, and normalize it all so that $X$ and $Y$ all lies between $[-1,1]$. Note that we determine the normalisation for the training set, and we apply the *same* normalization to the validation and test sets!

In [ ]:
from sklearn.model_selection import train_test_split
# Split the sets into a test and a "full" training set, picking randomly each sample
X_train_full, X_test, Y_train_full, Y_test = train_test_split(X_total, Y_total, random_state=42)

# Split the full training set into a training set and a validation set
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train_full, Y_train_full, random_state=42)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) # compute normalization and apply it
X_valid = scaler.transform(X_valid)   # apply normalization
X_test = scaler.transform(X_test)     # apply normalization

scalerY = StandardScaler()            # new normalization for Y
Y_train = scalerY.fit_transform(Y_train)
Y_valid = scalerY.transform(Y_valid)
Y_test = scalerY.transform(Y_test)


Check that now our sets are properly normalized

In [ ]:
plt.figure()
plt.plot(Y_train)

# 3- Make a neural network!

### a- import and initialize tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
# Init tensor flow
keras.backend.clear_session() 
np.random.seed(42)
tf.random.set_seed(42)

### b- create a neural network

The neural network will hae with 3 hidden layers, and a "relu" activation function in each layer. The final layer as two neurons, each corresponding 

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(256, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(2)
])
model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=1e-3))


### c- train it!

In [ ]:
history = model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid))

### d- test it!

Check how we're doing on the test set (note that the network has not seen the test data up to this point)

In [ ]:
mse_test = model.evaluate(X_test, Y_test)

# 4- Use the neural network

### a- Inference function

Define a function to infer the EMFs given a state vector. A state is dictionnary assumed to contain $\rho$ and $B_x$, $B_y$ along $z$ at a given instant

In [ ]:
def get_Emf(model,state):
    # construct the vector used by the model
    # assumes state contains logrho and everything
    # Create an input vector
    X=np.zeros([state[Xkeylist[0]].shape[0]-2,9])
    nvars=len(Xkeylist)
    # fill X
    nk=0 # number of the key
    for key in Xkeylist:
        pcentered=state[key][1:-1].flatten()
        pm1=state[key][0:-2].flatten()
        pp1=state[key][2:].flatten()
        X[:,nk]=pcentered
        X[:,nk+nvars]=pm1
        X[:,nk+2*nvars]=pp1
        nk=nk+1
    
    Xscaled = scaler.transform(X)

    # Call the neural network to make a guess for Y
    Yscaled = model.predict(Xscaled)

    # invert Y according to the scaler used above
    Y=scalerY.inverse_transform(Yscaled)
    Ex=Y[:,0]
    Ey=Y[:,1]
    return Ex,Ey 

### b- Use the inference function

Here we show how to use the inference function using the simulations data at $t_\mathrm{ref}=150$. We first build a state, call the get_emf function with the neural network and state, and finally plot the result

In [ ]:
tref=150
state={}
state['logrho']=avg['logrho'][:,tref]
state['bx']=avg['bx'][:,tref]
state['by']=avg['by'][:,tref]

Ex,Ey=get_Emf(model,state)
plt.figure()
plt.plot(avg['z'],avg['Ex'][:,tref],label='Simulation')
plt.plot(avg['z'][1:-1],Ex,label='Neural network')
plt.title(r'$E_x$')
plt.legend()

plt.figure()
plt.plot(avg['z'],avg['Ey'][:,tref],label='Simulation')
plt.plot(avg['z'][1:-1],Ey,label='Neural network')
plt.title(r'$E_y$')
plt.legend()